## Prepare Data + Model

In [22]:
# !cat examples/data/text_forward.txt

In [23]:
# !ls -al ./outputs/en.1-percent.elmo-bert-causal

In [4]:
import torch
from newlm.lm.elmo.modeling_elmo.elmo_head import ELMOBertLMHeadModel
from newlm.lm.elmo.lm_builder import ELMOLMBuilder
from transformers import BertConfig

#### Model

In [6]:
model = ELMOBertLMHeadModel.from_pretrained(
    "./outputs/en.1-percent.elmo-bert-causal"
) # use pre-trained model

In [13]:
model.eval()
print("Model in eval mode for consistency")

Model in eval mode for consistency


#### Data

In [10]:
%%capture

from newlm.utils.file_util import read_from_yaml
config_file = read_from_yaml('examples/configs/run.1-percent-elmo-bert-causal.yaml')

# lm builder (helper)
elmo_lm_builder = ELMOLMBuilder(
    model_config = config_file['lm']['model']['config'],
    tokenizer="./outputs/en.1-percent.elmo-bert-causal", # use pre-trained tokenizer
    model_type="bert-causal-elmo"
)

# dataset-forward
train_path = "./examples/data/text_forward.txt"
ds_f = elmo_lm_builder._get_dataset(train_path)

2021-11-11 19:28:13.210 | INFO     | newlm.lm.elmo.lm_builder:_get_dataset:142 - Constructing roBERTa style dataset


In [11]:
# trainer (helper)
from transformers import TrainingArguments, Trainer
args = TrainingArguments(output_dir="tmpout",**config_file['lm']['hf_trainer']['args'])

# dataloader-forward
trainer = Trainer(model=model, args=args, data_collator=elmo_lm_builder.data_collator, train_dataset=ds_f,)
dl_f = trainer.get_train_dataloader() # Data Loader-forward

max_steps is given, it will override any value given in num_train_epochs


In [12]:
batch_f = next(iter(dl_f))
batch_f['input_ids'].shape

torch.Size([1, 272])

## Sanity Check

In [15]:
# batch_f

In [16]:
import torch

# reverse input
batch_f_input = torch.clone(batch_f['input_ids'])
batch_f_rev_input = torch.cat(
    (
        batch_f_input[0][0:1],
        torch.flip(batch_f_input[0][1:-1], [0]),
        batch_f_input[0][-1:]
    )
)
batch_f_rev_input = batch_f_rev_input.reshape(1,-1)

# reverse labels
batch_f_labels = torch.clone(batch_f['labels'])
batch_f_rev_labels = torch.cat(
    (
        batch_f_labels[0][0:1],
        torch.flip(batch_f_labels[0][1:-1], [0]),
        batch_f_labels[0][-1:]
    )
)
batch_f_rev_labels = batch_f_rev_labels.reshape(1,-1)

batch_rev = batch_f.copy()
batch_rev['input_ids'] = batch_f_rev_input
batch_rev['labels'] = batch_f_rev_labels

In [17]:
tokens_f = elmo_lm_builder.tokenizer.convert_ids_to_tokens(batch_f['input_ids'][0])
tokens_f_rev = elmo_lm_builder.tokenizer.convert_ids_to_tokens(batch_rev['input_ids'][0])

import pandas as pd
pd.DataFrame({"forward": tokens_f, "reverse": tokens_f_rev})

,forward,reverse
0,[CLS],[CLS]
1,R,.
2,##ø,periods
3,##d,##an
4,##berg,republic
...,...,...
267,republic,##berg
268,##an,##d
269,periods,##ø
270,.,R


In [18]:
batch_f['input_ids'].shape, batch_rev['input_ids'].shape

(torch.Size([1, 272]), torch.Size([1, 272]))

In [19]:
res = model(**batch_f) # forward

l2r_loss tensor(4.8444, grad_fn=<NllLossBackward>)
r2l_loss tensor(8.5702, grad_fn=<NllLossBackward>)


In [20]:
res = model(**batch_rev) # reverse

l2r_loss tensor(9.3590, grad_fn=<NllLossBackward>)
r2l_loss tensor(8.4301, grad_fn=<NllLossBackward>)
